In [1]:
import numpy as np
import math

In [2]:
class MinHeap:
    def __init__(self):
        self.nodes = []

    def get_min(self):
        return self.nodes[0]

    def pop_min(self):
        if len(self.nodes) == 0:
            return -1
        elif len(self.nodes) == 1:
            m = self.nodes[0]
            self.nodes = []
            return m
        else:
            m = self.nodes[0]
            l = self.nodes[-1]
            self.nodes = self.nodes[:-1]
            self.nodes[0] = l
            # perculate down
            p = 0
            l = 2*p + 1
            r = 2*p + 2
            while r < len(self.nodes) and self.nodes[p] > min(self.nodes[l], self.nodes[r]):
                if self.nodes[l] > self.nodes[r]:
                    t = self.nodes[p]
                    self.nodes[p] = self.nodes[r]
                    self.nodes[r] = t
                    p = r
                else:
                    t = self.nodes[p]
                    self.nodes[p] = self.nodes[l]
                    self.nodes[l] = t
                    p = l
                l = 2*p + 1
                r = 2*p + 2
            # accounting for odd case where there is not a left and right
            if len(self.nodes) % 2 == 0 and self.nodes[-1] < self.nodes[(len(self.nodes) - 2) // 2]:
                # swap values
                t = self.nodes[-1]
                self.nodes[-1] = self.nodes[(len(self.nodes) - 2) // 2]
                self.nodes[(len(self.nodes) - 2) // 2] = t
            return m

    def insert(self, x):
        self.nodes.append(x)
        i = len(self.nodes)-1
        p = (i - 1) // 2
        while i > 0 and self.nodes[p] > self.nodes[i]:
            t = self.nodes[p]
            self.nodes[p] = self.nodes[i]
            self.nodes[i] = t
            i = p
            p = (i - 1) // 2


In [3]:
mh = MinHeap()

n = np.random.randint(1000,size=50)
for i in range(50):
    mh.insert(int(n[i]))

In [4]:
print(mh.nodes)

[36, 46, 46, 193, 60, 59, 367, 301, 316, 188, 67, 105, 437, 752, 480, 574, 503, 589, 644, 775, 553, 463, 79, 479, 402, 464, 699, 953, 765, 854, 935, 775, 920, 865, 887, 669, 833, 883, 708, 965, 853, 957, 799, 940, 888, 916, 750, 826, 640, 506]


In [5]:
sorted(mh.nodes) == [mh.pop_min() for i in range(50)]

True

In [90]:
class PairingHeap:
    def __init__(self, x):
        self.key = x
        self.child = None
        self.left = None
        self.right = None
    
    def meld(self, other):
        """
        Meld other and self. You should only meld 2 roots.
        Parameters:
            other(PairingHeap): PairingHeap to be melded with self
        Return:
            (PairingHeap): resultant pairing heap
        """
        if self.key >= other.key:
            # set other as left child of self
            if self.child is not None:
                self.child.left = other
            other.right = self.child
            self.child = other
            # left of the left-most sibling is the parent
            other.left = self
            return self
        else:
            # set self as left child of other
            if other.child is not None:
                other.child.left = self
            self.right = other.child
            other.child = self
            self.left = other
            return other

    def insert(self, x):
        # probably will not be used if you want to 
        new_tree = PairingHeap(x)
        return self.meld(new_tree)

    def increase_key(self, n, i):
        n.key += i
        # remove from sibling linked list
        if n.right is not None:
            n.right.left = n.left
        if n.left is not None:
            if n.left.child == n:
                n.left.child = n.right
            else:
                n.left.right = n.right
        # meld self with the node with increased key
        return self.meld(n)

    def remove_max(self):
        
        result = self.key

        # first pass
        children = [self.child]
        while children[-1] is not None:
            if children[-1].right is not None:
                s = children[-1].right
                t = children[-1].right.right
                s.right = None
                s.left = None
                children[-1].right = None
                children[-1].left = None
                children[-1] = children[-1].meld(s)
                children.append(t)
            else:
                children.append(None)
                break
        # remove None from end of list
        children = children[:-1]

        # second pass
        while len(children) > 1:
            children = children[:-2] + [children[-1].meld(children[-2])]
        # If the node is the last in the PairingHeap, then children will have length 0
        # return None as the new PairingHeap
        children.append(None)
        return result, children[0]

    def dfs(self):
        # print key in depth first manner
        if self.child is not None:
            self.child.dfs()
        print(self.key)
        if self.right is not None:
            self.right.dfs()

    def __str__(self):
        return f"{self.key}"

In [91]:
np.random.seed(1)
n = np.random.randint(100,size=5)
ph = PairingHeap(n[0])
nodes = [ph]
for i in range(1,5):
    nodes.append(PairingHeap(n[i]))
    ph = ph.meld(nodes[-1])

In [96]:
[n.__str__() for n in nodes]

['37', '12', '82', '9', '75']

In [93]:
ph.dfs()

9
12
37
72
75


In [94]:
ph = ph.increase_key(nodes[2], 10)

In [95]:
ph.dfs()

75
9
12
37
82


In [104]:
r, ph = ph.remove_max()

In [105]:
r

np.int64(37)

In [106]:
ph.dfs()

9
12


In [54]:
l = []
for i in range(5):
    r, ph = ph.remove_max()
    l.append(r)
l

[75, 72, np.int64(37), 12, 9]

In [56]:
ph